In [ ]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from scipy.integrate import trapz
from scipy.io import loadmat
import scipy.io

from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM
import tensorflow as tf

In [ ]:
# Input Parameters Initialization

Lx = 4 * np.pi
Ly = 2
Lz = 2 * np.pi

nx = ny = nz = 21

Re = 400  # Reynold's Number
a = 2 * np.pi / Lx  # alpha
b = np.pi / 2.  # beta
g = 2 * np.pi / Lz  # gamma
params = [a, b, g, Re]

x = np.linspace(0, Lx, nx)
y = np.linspace(-1, 1, ny)
z = np.linspace(0, Lz, nz)
time = np.linspace(0, 4000, 4001)  # time array
seqLen = 10

LOOK_BACK = 10
NUM_EPOCHS = 10

HIDDEN_UNITS = 90

In [ ]:
# Logic to discard datasets that laminarize with time
def check_for_laminarization(sol, threshold):
    sol = sol.loc[:, 4].to_numpy()
    for i in range(100, len(sol)):
        if np.abs(sol[i] - sol[i-100]) < threshold:
            return 1
    return 0


def MoehlisCoefficientsGenerator(y0, t, params):

    # setting up of initial coefficient values
    a1, a2, a3, a4, a5, a6, a7, a8, a9 = y0

    # setting up of parameters
    params = a, b, g, Re

    # contstants' definition
    k_ag = np.sqrt(np.square(a) + np.square(g))
    k_bg = np.sqrt(np.square(b) + np.square(g))
    k_ab = np.sqrt(np.square(a) + np.square(b))
    k_abg = np.sqrt(np.square(a) + np.square(b) + np.square(g))

    # calculation of differential coefficients - Equations as it is from Srinivasan's paper
    da1dt = (b**2 / Re) * (1 - a1) - np.sqrt(3/2) * \
        ((b*g/k_abg) * a6 * a8 - (b*g/k_bg) * a2 * a3)

    da2dt = -(4*b**2/3 + g**2)*(a2/Re) + ((5*np.sqrt(2)*g**2)/(3*np.sqrt(3)*k_ag))*a4*a6 - (g**2/(np.sqrt(6)*k_ag))*a5*a7 - \
        ((a*b*g)/(np.sqrt(6)*k_ag*k_abg))*a5*a8 - np.sqrt(3/2)*((b*g)/(k_bg))*a1*a3 - \
        np.sqrt(3/2)*((b*g)/(k_bg))*a3*a9

    da3dt = -(k_bg**2/Re)*a3 + ((2*a*b*g)/(np.sqrt(6)*k_ag*k_bg))*(a4*a7+a5*a6) + \
        ((b**2*(3*a**2+g**2)-3*g**2*k_ag**2)/(np.sqrt(6)*k_ag*k_bg*k_abg))*a4*a8

    da4dt = -((3*a**2+4*b**2)/(3*Re))*a4 - (a/np.sqrt(6))*a1*a5 - ((10*a**2)/(3*np.sqrt(6)*k_ag))*a2*a6 - \
        np.sqrt(3/2)*((a*b*g)/(k_ag*k_bg))*a3*a7 - np.sqrt(3/2)*((a**2*b**2)/(k_ag*k_bg*k_abg))*a3*a8 - \
        (a/np.sqrt(6))*a5*a9

    da5dt = -(k_ab**2/Re)*a5 + (a/np.sqrt(6))*a1*a4 + (a**2/(np.sqrt(6)*k_ag))*a2*a7 - \
        (a*b*g/(np.sqrt(6)*k_ag*k_abg))*a2*a8 + (a/np.sqrt(6))*a4*a9 + \
        ((2*a*b*g)/(np.sqrt(6)*k_ag*k_bg))*a3*a6

    da6dt = -((3*a**2+4*b**2+3*g**2)/(3*Re))*a6 + (a/np.sqrt(6))*a1*a7 + (np.sqrt(3/2)*b*g/(k_abg))*a1*a8 + \
        (10*(a**2-g**2)/(3*np.sqrt(6)*k_ag))*a2*a4 - (2*np.sqrt(2/3)*a*b*g/(k_ag*k_bg))*a3*a5 + \
        (a/np.sqrt(6))*a7*a9 + (np.sqrt(3/2)*b*g/k_abg)*a8*a9

    da7dt = -(k_abg**2/Re)*a7 - a*(a1*a6+a6*a9)/np.sqrt(6) + ((g**2-a**2)/(np.sqrt(6)*k_ag))*a2*a5 + \
        ((a*b*g)/(np.sqrt(6)*k_ag*k_bg))*a3*a4

    da8dt = -(k_abg**2/Re)*a8 + ((2*a*b*g)/(np.sqrt(6)*k_ag*k_abg))*a2*a5 + \
        (g**2*(3*a**2-b**2+3*g**2)/(np.sqrt(6)*k_ag*k_bg*k_abg))*a3*a4

    da9dt = -(9*b**2/Re)*a9 + (np.sqrt(3/2)*b*g/k_bg) * \
        a2*a3 - (np.sqrt(3/2)*b*g/k_abg)*a6*a8

    # returning the differential coefficients in a list format
    func = [da1dt, da2dt, da3dt, da4dt, da5dt, da6dt, da7dt, da8dt, da9dt]
    return func


def generate_datasets(num_of_datasets, threshold=1e-6):

    count = 0
    datasets = []
    init_params = []

    while count < num_of_datasets:
        a4 = 0.1*np.round(np.random.uniform(0, 1), 3)
        y0 = [1, 0.07066, -0.07076, a4, 0, 0, 0, 0, 0]

        time = np.linspace(0, 4000, 4001)

        sol = odeint(MoehlisCoefficientsGenerator, y0, time, args=(params,))
        if check_for_laminarization(pd.DataFrame(sol), threshold):
            continue
        else:
            count = count + 1
            datasets.append(sol)
            init_params.append(y0)

    datasets = np.array(datasets)
    init_params = np.array(init_params)
    return datasets, init_params

# Utility Function: Generating sequences


def generate_sequences(A, model):
    seqNoList = range(0, 10)
    seqLen = 10

    if not os.path.exists(os.path.join(os.getcwd(), "Sequences")):
        os.mkdir(os.path.join(os.getcwd(), "Sequences"))

    saveFilename = "./Sequences/series_"

    for seqNo in seqNoList:
        print(seqNo + 1)

        testSeq = A[seqNo:seqNo+1]
        predSeq = testSeq[:, :seqLen]

        for i in np.arange(testSeq.shape[1] - seqLen):
            nextState = model.predict(predSeq[:, i:i+seqLen], verbose=0)
            predSeq = np.concatenate((predSeq, [nextState]), axis=1)

        testSeq = testSeq.reshape(-1, 9)
        predSeq = predSeq.reshape(-1, 9)

        save_filename = f"{saveFilename}{seqNo + 1}.npz"
        np.savez(save_filename, testSeq=testSeq, predSeq=predSeq)


def magnitude_vs_time_plot(actual_data, predicted_data):

    actual_sol = actual_data
    predicted_sol = predicted_data

    fig1 = plt.figure(figsize=(8, 8))

    time = np.linspace(0, 4000, 4001)

    a1, a2, a3, a4, a5, a6, a7, a8, a9 = [
        1, 0.07066, -0.07076, 0.0996, 0, 0, 0, 0, 0]

    title = rf"$a_1 = {a1}, a_2 = {a2}, a_3 = {a3}, a_4 = {a4}, a_5 = {a5}, a_6 = {a6}, a_7 = {a7}, a_8 = {a8}, a_9 = {a9}$"

    fig1.suptitle(title)

    plt.subplot(5, 2, 1)
    plt.plot(time, actual_sol[:, 0], '--b', label=r"Actual $a_1$")
    plt.plot(time, predicted_sol[:, 0], '-r', label=r"Predicted $a_1$")
    plt.legend()

    plt.subplot(5, 2, 2)
    plt.plot(time, actual_sol[:, 1], '--b', label=r"Actual $a_2$")
    plt.plot(time, predicted_sol[:, 1], '-r', label=r"Predicted $a_2$")
    plt.legend()

    plt.subplot(5, 2, 3)
    plt.plot(time, actual_sol[:, 2], '--b', label=r"Actual $a_3$")
    plt.plot(time, predicted_sol[:, 2], '-r', label=r"Predicted $a_3$")
    plt.legend()

    plt.subplot(5, 2, 4)
    plt.plot(time, actual_sol[:, 3], '--b', label=r"Actual $a_4$")
    plt.plot(time, predicted_sol[:, 3], '-r', label=r"Predicted $a_4$")
    plt.legend()

    plt.subplot(5, 2, 5)
    plt.plot(time, actual_sol[:, 4], '--b', label=r"Actual $a_5$")
    plt.plot(time, predicted_sol[:, 4], '-r', label=r"Predicted $a_5$")
    plt.legend()

    plt.subplot(5, 2, 6)
    plt.plot(time, actual_sol[:, 5], '--b', label=r"Actual $a_6$")
    plt.plot(time, predicted_sol[:, 5], '-r', label=r"Predicted $a_6$")
    plt.legend()

    plt.subplot(5, 2, 7)
    plt.plot(time, actual_sol[:, 6], '--b', label=r"Actual $a_7$")
    plt.plot(time, predicted_sol[:, 6], '-r', label=r"Predicted $a_7$")
    plt.legend()

    plt.subplot(5, 2, 8)
    plt.plot(time, actual_sol[:, 7], '--b', label=r"Actual $a_8$")
    plt.plot(time, predicted_sol[:, 7], '-r', label=r"Predicted $a_8$")
    plt.legend()

    plt.subplot(5, 2, 9)
    plt.plot(time, actual_sol[:, 8], '--b', label=r"Actual $a_9$")
    plt.plot(time, predicted_sol[:, 8], '-r', label=r"Predicted $a_9$")
    plt.legend()

    plt.show()

In [ ]:
u2x = np.zeros((len(x), len(y), len(z)))
u1x = np.zeros((len(x), len(y), len(z)))
u3y = np.zeros((len(x), len(y), len(z)))
u3z = np.zeros((len(x), len(y), len(z)))
u4z = np.zeros((len(x), len(y), len(z)))
u5z = np.zeros((len(x), len(y), len(z)))
u6x = np.zeros((len(x), len(y), len(z)))
u6z = np.zeros((len(x), len(y), len(z)))
u7x = np.zeros((len(x), len(y), len(z)))
u7z = np.zeros((len(x), len(y), len(z)))
u8x = np.zeros((len(x), len(y), len(z)))
u8y = np.zeros((len(x), len(y), len(z)))
u8z = np.zeros((len(x), len(y), len(z)))
u9x = np.zeros((len(x), len(y), len(z)))

for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            u1x[i, j, k] = (np.sqrt(2)*np.sin(np.pi*y[j]/2.))
            u2x[i, j, k] = (4/np.sqrt(3)) * \
                (np.cos(np.pi*y[j]/2))**2*np.cos(g*z[k])
            u3y[i, j, k] = (2./np.sqrt(4*g**2+np.pi**2))*2*g * \
                np.cos(np.pi*y[j]/2)*np.cos(g*z[k])
            u3z[i, j, k] = (2./np.sqrt(4*g**2+np.pi**2))*np.pi * \
                np.sin(np.pi*y[j]/2)*np.sin(g*z[k])
            u4z[i, j, k] = (4/np.sqrt(3))*np.cos(a*x[i]) * \
                (np.cos(np.pi*y[j]/2))**2
            u5z[i, j, k] = 2*np.sin(a*x[i])*np.sin(np.pi*y[j]/2)
            u6x[i, j, k] = (4*np.sqrt(2)/np.sqrt(3*(a**2+g**2)))*(-g) * \
                np.cos(a*x[i])*(np.cos(np.pi*y[j]/2))**2*np.sin(g*z[k])
            u6z[i, j, k] = (4*np.sqrt(2)/np.sqrt(3*(a**2+g**2)))*a * \
                np.sin(a*x[i])*(np.cos(np.pi*y[j]/2))**2*np.cos(g*z[k])
            u7x[i, j, k] = (2*np.sqrt(2)/np.sqrt(a**2+g**2))*g * \
                np.sin(a*x[i])*np.sin(np.pi*y[j]/2)*np.sin(g*z[k])
            u7z[i, j, k] = (2*np.sqrt(2)/np.sqrt(a**2+g**2))*a * \
                np.cos(a*x[i])*np.sin(np.pi*y[j]/2)*np.cos(g*z[k])
            u8x[i, j, k] = (2*np.sqrt(2)/(np.sqrt((a**2+g**2)*(4*a**2+4*g**2+np.pi**2)))
                            )*np.pi*a*np.sin(a*x[i])*np.sin(np.pi*y[j]/2)*np.sin(g*z[k])
            u8y[i, j, k] = (2*np.sqrt(2)/(np.sqrt((a**2+g**2)*(4*a**2+4*g**2+np.pi**2)))) * \
                2*(a**2+g**2)*np.cos(a*x[i]) * \
                np.cos(np.pi*y[j]/2)*np.sin(g*z[k])
            u8z[i, j, k] = (2*np.sqrt(2)/(np.sqrt((a**2+g**2)*(4*a**2+4*g**2+np.pi**2)))) * \
                (-np.pi*g)*np.cos(a*x[i])*np.sin(np.pi*y[j]/2)*np.cos(g*z[k])
            u9x[i, j, k] = np.sqrt(2)*np.sin(3*np.pi*y[j]/2)

In [ ]:
def uxbar_vs_y_plot(U_X_pred, U_X_true):

    ux_bar_pred = np.zeros(nx)
    ux_bar_true = np.zeros(nx)
    nt = 200
    for j in range(ny):
        ux_bar_pred[j] = np.sum(U_X_pred[:, j, :], axis=(0, 1))
        ux_bar_true[j] = np.sum(U_X_true[:, j, :], axis=(0, 1))
    ux_bar_pred *= (1/(nx*nz))
    ux_bar_true *= (1/(nx*nz))

    plt.plot(ux_bar_pred, y, 'red', label='ux_bar_pred', linewidth=2)
    plt.plot(ux_bar_true, y, linestyle='dotted',
             color='blue', label='ux_bar_true', linewidth=2)
    plt.title('ux_bar vs. y')
    plt.xlabel('u̅')
    plt.ylabel('y')
    plt.legend()
    plt.show()


def quiver_contour_plots_xavg(ux, uy, uz, t):

    fig2, ax = plt.subplots(figsize=(10, 5))
    Z, Y = np.meshgrid(z, y)
    plt.xlabel("$z$")
    plt.ylabel("$y$")

    ux_avg = np.mean(ux, axis=0).reshape(ny, nz)
    uy_avg = np.mean(uy, axis=0).reshape(ny, nz)
    uz_avg = np.mean(uz, axis=0).reshape(ny, nz)

    contour = plt.contourf(Z, Y, ux_avg, cmap='jet')
    plt.colorbar(contour, label='ux_avg')

    plt.quiver(Z, Y, uz_avg, uy_avg, color='k')

    plt.title(r"$t = {0}$".format(t), fontsize=15)
    plt.show()


def calculate_components_new(a):
    ux = a[0]*u1x + a[1]*u2x + a[5]*u6x + a[6]*u7x + a[7]*u8x + a[8]*u9x
    uy = a[2]*u3y + a[7]*u8y
    uz = a[2]*u3z + a[3]*u4z + a[4]*u5z + a[5]*u6z + a[6]*u7z + a[7]*u8z
    return ux, uy, uz

In [ ]:
datasets, init_params = generate_datasets(12)

init_params.shape

train_init_params = init_params[:10]
test_init_params = init_params[10:]

train_data = datasets[:10]
test_data = datasets[10:]

np.save('train_init_params.npy', train_init_params)
np.save('test_init_params.npy', test_init_params)

np.save('train_data.npy', train_data)
np.save('test_data.npy', test_data)

# ## Model Training

trained_model = False

if trained_model == True:
    model = load_model('path/to/trained/model')
else:
    NUM_EPOCHS = 10
    model = Sequential()

    model.add(LSTM(HIDDEN_UNITS,
                   input_shape=(LOOK_BACK, 9),
                   kernel_initializer='glorot_normal',
                   return_sequences=False))

    model.add(Dense(9, activation='tanh'))
    model.compile(optimizer='adam', loss='mean_squared_error')

A = np.load('./train_data.npy')

# The number of training samples
nSamples = A.shape[0]*(A.shape[1]-seqLen)

# Initialize the training inputs and outputs using empty arrays
X = np.empty([nSamples, seqLen, A.shape[2]])
Y = np.empty([nSamples, A.shape[2]])

# Fill the input and output arrays with data
k = 0
for i in np.arange(A.shape[0]):
    for j in np.arange(A.shape[1]-seqLen):
        X[k] = A[i, j:j+seqLen]
        Y[k] = A[i, j+seqLen]
        k = k + 1

score = model.fit(X, Y, batch_size=32, epochs=NUM_EPOCHS,
                  verbose=1, validation_split=0.20, shuffle=True)


# Save losses
lossHistory = score.history['loss']
valLossHistory = score.history['val_loss']

# Save the losses
with open('loss_history.txt', 'w') as file:
    for loss, val_loss in zip(lossHistory, valLossHistory):
        file.write(f'Train Loss: {loss}\tVal loss: {val_loss}\n')

# Save the model
model.save('trained_model.keras')

In [ ]:
generate_sequences(test_data, model)
seq = np.load(os.path.join(os.getcwd(), 'Sequences/series_1.npz'))
testSequences = seq['testSeq']
predSequences = seq['predSeq']

testSequences.shape

test_init_params[0]

atest_ = testSequences
apred_ = predSequences

start_t = 500
end_t = 500

for ti in range(start_t, end_t+1):
    atest = atest_[ti, :]
    apred = apred_[ti, :]

    ux_test, uy_test, uz_test = calculate_components_new(atest)
    ux_pred, uy_pred, uz_pred = calculate_components_new(apred)

    quiver_contour_plots_xavg(ux_test, uy_test, uz_test, ti)
    uxbar_vs_y_plot(ux_pred, ux_test)

magnitude_vs_time_plot(testSequences, predSequences)

atest_ = testSequences

start_t = 500
end_t = start_t

for ti in range(start_t, end_t+1):
    atest = atest_[ti, :]

    ux_test, uy_test, uz_test = calculate_components_new(atest)

    quiver_contour_plots_xavg(ux_test, uy_test, uz_test, ti)

atest_ = testSequences

start_t = 2000
end_t = start_t

for ti in range(start_t, end_t+1):
    atest = atest_[ti, :]

    ux_test, uy_test, uz_test = calculate_components_new(atest)

    quiver_contour_plots_xavg(ux_test, uy_test, uz_test, ti)